In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
from data_extraction import *
from resp_signal_extraction import *
from rr_extration import *
from sklearn.preprocessing import MinMaxScaler
import re
import tensorflow as tf
import pickle as pkl
from tf_model import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
import evidential_deep_learning as edl
import matplotlib.pyplot as plt

In [ ]:
input_conf = 'confc'

In [ ]:

with open('output','rb') as f:
    output_data = pkl.load(f)

with open('input','rb') as f:
    input_data = pkl.load(f)

with open('raw_signal.pkl','rb') as f:
    raw_data = pkl.load(f)

input_data = input_data.reshape(input_data.shape[0],input_data.shape[-1],input_data.shape[1])
raw_data = raw_data.reshape(raw_data.shape[0],raw_data.shape[-1],raw_data.shape[1])
annotation = pd.read_pickle('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/annotation.pkl')
reference_rr = (annotation['Reference_RR'].values).reshape(-1,1)

tensor_input = tf.convert_to_tensor(input_data, dtype = 'float32')
tensor_output = tf.convert_to_tensor(output_data, dtype = 'float32')
tensor_ref_rr = tf.convert_to_tensor(reference_rr, dtype = 'float32')
tensor_raw_data = tf.convert_to_tensor(raw_data, dtype = 'float32')
training_ids = annotation['patient_id'] < 13

x_train_data = tensor_input[tf.convert_to_tensor(training_ids.values)]
x_test_data = tensor_input[tf.convert_to_tensor(~(training_ids.values))]
x_train_ref_rr = tensor_ref_rr[tf.convert_to_tensor(training_ids.values)]
x_test_ref_rr = tensor_ref_rr[tf.convert_to_tensor(~(training_ids.values))]
x_train_raw_sig = tensor_raw_data[tf.convert_to_tensor(training_ids.values)]
x_test_raw_sig = tensor_raw_data[tf.convert_to_tensor(~(training_ids.values))]

y_train_data = tensor_output[tf.convert_to_tensor(training_ids.values)]
y_test_data = tensor_output[tf.convert_to_tensor(~(training_ids.values))]


In [ ]:
if input_conf == 'confc':
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , y_train_data))
    train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , y_test_data))
    test_dataset = test_dataset.batch(128)

if input_conf == 'confd':
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , y_train_data, x_train_ref_rr))
    train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , y_test_data, x_test_ref_rr))
    test_dataset = test_dataset.batch(128)
    
if input_conf == 'confb':
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , x_train_ref_rr))
    train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , x_test_ref_rr))
    test_dataset = test_dataset.batch(128)

if input_conf == 'confe':
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw_sig , y_train_data))
    train_dataset = train_dataset.shuffle(len(x_train_raw_sig)).batch(128)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw_sig , y_test_data))
    test_dataset = test_dataset.batch(128)

if input_conf == 'confa':
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw_sig , x_train_ref_rr))
    train_dataset = train_dataset.shuffle(len(x_train_raw_sig)).batch(128)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw_sig , x_test_ref_rr))
    test_dataset = test_dataset.batch(128)

if input_conf == 'conff':
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_raw_sig , y_train_data, x_train_ref_rr))
    train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_raw_sig , y_test_data, x_test_ref_rr))
    test_dataset = test_dataset.batch(128)
    


In [ ]:
if input_conf == 'confc':
    model_input_shape = (128,3)
    model  = BRUnet(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confc/best_modellr_0.0001__coeff_0.01.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step,(x_batch_test , y_batch_test) in enumerate(test_dataset):
        output = model(x_batch_test)
        test_loss = loss_fn(y_batch_test,output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#-----------------------------------------------------------------------------------------------------------------
if input_conf == 'confd':
    model_input_shape = (128,3)
    model  = BRUnet_Multi_resp(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confd/best_model_lr_0.0001__coeff_0.01.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    final_output_rr = tf.convert_to_tensor([])
    for step , (x_batch_test,y_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
        test_output,test_out_rr = model(x_batch_test)
        test_loss_resp = loss_fn(y_batch_test  , test_output)
        test_loss_rr = loss_fn(x_batch_test_ref_rr , test_out_rr)
        test_loss = test_loss_resp + test_loss_rr
        if step == 0:
            final_output = test_output
            final_output_rr = test_out_rr
        else:
            final_output = tf.concat([final_output , test_output] , axis = 0)
            final_output_rr = tf.concat([final_output_rr , test_out_rr] , axis = 0)
        test_loss_list.append(test_loss)
            
#------------------------------------------------------------------------------------------------------------------
if input_conf == 'confb':
    model_input_shape = (128,3)
    model  = BRUnet_Encoder(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,128,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confb/best_model_lr_0.0001__coeff_0.01.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
        output = model(x_batch_test)
        test_loss = loss_fn(x_batch_test_ref_rr , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#--------------------------------------------------------------------------------------------------------------------
if input_conf == 'confe':
    model_input_shape = (2048,3)
    model  = BRUnet_raw(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confe/best_model_lr_0.0001__coeff_0.01.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test_raw,y_batch_test) in enumerate(test_dataset):
        output = model(x_batch_test_raw)
        test_loss = loss_fn(y_batch_test , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
        
#-------------------------------------------------------------------------------------------------------------------
if input_conf == 'confa':
    model_input_shape = (2048,3)
    model  = BRUnet_raw_encoder(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/confa/best_model_lr_0.0001__coeff_0.01.h5')
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    for step , (x_batch_test_raw, x_batch_test_ref_rr) in enumerate(test_dataset):
        output = model(x_batch_test_raw)
        test_loss = loss_fn(x_batch_test_ref_rr , output)
        if step == 0:
            final_output = output
        else:
            final_output = tf.concat([final_output , output] , axis = 0)
        test_loss_list.append(test_loss)
#-------------------------------------------------------------------------------------------------------------------
if input_conf == 'conff':
    model_input_shape = (2048,3)
    model  = BRUnet_raw_multi(model_input_shape)
    loss_fn = Huber()
    model(tf.ones((128,2048,3)))
    model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/conff/best_model_lr_0.0001__coeff_0.01.h5')        
    test_loss_list = []
    final_output = tf.convert_to_tensor([])
    final_output_rr = tf.convert_to_tensor([])
    for step , (x_batch_test_raw , y_batch_test , x_batch_test_ref_rr) in enumerate(test_dataset):
        test_output,test_out_rr = model(x_batch_test_raw)
        test_loss_resp = loss_fn(y_batch_test  , test_output)
        test_loss_rr = loss_fn(x_batch_test_ref_rr , test_out_rr)
        test_loss = test_loss_resp + test_loss_rr
        if step == 0:
            final_output = test_output
            final_output_rr = test_out_rr
        else:
            final_output = tf.concat([final_output , test_output] , axis = 0)
            final_output_rr = tf.concat([final_output_rr , test_out_rr] , axis = 0)
        test_loss_list.append(test_loss)   

In [ ]:

    
model_input_shape = (2048,3)
model  = BRUnet_raw_multi(model_input_shape)
optimizer = Adam(learning_rate = 1e-4)
loss_fn = Huber()
model(tf.ones((128,2048,3)))
model.load_weights('/media/acrophase/Sentinel_1/charan/BR_Uncertainty/DL_BASED_METHOD/SAVED_MODELS/conff/best_model_lr_0.0001__coeff_0.01.h5')
test_loss_list = []
final_output = tf.convert_to_tensor([])
final_output_rr = tf.convert_to_tensor([])

for step , (x_batch_train_raw,y_batch_test,x_batch_test_ref_rr) in enumerate(test_dataset):
    output,out_4 = model(x_batch_train_raw)
    test_loss_resp = loss_fn(y_batch_test  , output)
    test_loss_rr = loss_fn(x_batch_test_ref_rr , out_4)
    test_loss = test_loss_resp + test_loss_rr
    print(tf.shape(output))
    if step == 0:
        final_output = output
        final_output_rr = out_4
    else:
        final_output = tf.concat([final_output , output] , axis = 0)
        final_output_rr = tf.concat([final_output_rr , out_4] , axis = 0)
    test_loss_list.append(test_loss)
    

In [ ]:
final_output = final_output.numpy()
final_output_rr = final_output_rr.numpy()

In [ ]:
final_output = final_output.reshape(final_output.shape[0],final_output.shape[1])
final_output_rr = final_output_rr.reshape(final_output_rr.shape[0],final_output_rr.shape[1])

In [ ]:
final_output_rr

In [ ]:
x_test_ref_rr

In [ ]:
err = np.abs(x_test_ref_rr - final_output_rr)

In [ ]:
final_output

In [ ]:
arr = final_output[123]

In [ ]:
plt.plot(arr)
plt.grid(True)
plt.show()

In [ ]:
arr